In [23]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
import glob, os, re, json
from scipy.stats import linregress
import statistics
def sort_names(image_files):
    file_names = [os.path.basename(string) for string in image_files]
    times = [(float(re.findall("\d+\.\d+", string)[0]), string) for string in file_names]
    times = sorted(times, key=lambda x: x[0])
    print(f"Time: {times[0][0]} -- {times[-1][0]}")
    image_files = [t[1] for t in times]
    return image_files

def give_time(file):
    filename = os.path.basename(file)
    time = re.findall("\d+\.\d+", filename)[0]
    return time

def plot_graph(list_x, list_y, names, xtitle, ytitle, image_name, list_x_fill=[], list_y_fill=[], mode=[], \
               dash=['solid', 'dot', 'dash', 'longdash'], \
               colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
               marker_size=15, xrange =[], yrange = [], vertical=None, \
               marker_style = ['circle', 'triangle-up', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
               width=1000, height=500, path='./', yanchor='center', y0_anchor=0.01, xanchor='left', x0_anchor=0.3, logx=False, logy=False):
    if mode == []:
        for i in range(len(list_x)):
            mode.append('lines+markers')


    while len(marker_style) < len(list_x):
        marker_style[:] = marker_style[:] + marker_style[:]
    figborderlinesize = 0.7
    legborderlinesize = 0.7
    yaxis = dict(
        tickfont = dict(
            family = 'Times New Roman',
            size = 20,
            color = 'black'
        ),
        titlefont = dict(
            family = 'Times New Roman',
            size = 25,
            color = 'black'
        )
    )
    if logy:
        yaxis['type'] = "log"
    xaxis = dict(
        tickfont = dict(
            family = 'Times New Roman',
            size = 20,
            color = 'black'
        ),
        titlefont = dict(
            family = 'Times New Roman',
            size = 25,
            color = 'black'
        )
    )
    if logx:
        xaxis['type'] = "log"

    axis_style = dict(showline=True, gridwidth=1, gridcolor='lightgrey', linewidth=figborderlinesize, linecolor='black', mirror=True, ticks='outside', tickfont = dict(family = 'Times New Roman', size = 20, color = 'black'))
    bg_style = {'plot_bgcolor': 'rgba(255, 255, 255, 1)', 'paper_bgcolor': 'rgba(255, 255, 255, 1)',}


    fig = go.Figure()
    k = len(list_x)
    n_fill = len(list_x_fill)
    if len(list_x_fill) == 2 and len(list_y_fill) == 2:
        fig.add_trace(go.Scatter(x=list_x_fill[1], y=list_y_fill[1], name=names[k+1], mode='lines', fillcolor='blueviolet', line_color='blueviolet', fill='tozeroy')) # fill to trace0 y
        fig.add_trace(go.Scatter(x=list_x_fill[0], y=list_y_fill[0], name=names[k], mode='lines', fillcolor='lightsteelblue',     line_color='indigo', fill='tozeroy')) # fill down to xaxis
    for i,x in enumerate(list_x):
        print('Plot curve number:', i)
        y = np.asarray(list_y[i])
        fig.add_trace(go.Scatter(x=x, y=y, name=names[i],
                                 mode=mode[i],
                                 marker=dict(
                                     size=marker_size,
                                     line=dict(width=1)
                                 ),
                                 marker_symbol=marker_style[i],
                                 line=dict(width=2, dash=dash[i]),
                                 textfont=dict(
                                     family="Times New Roman",
                                     size=18,
                                     color="LightSeaGreen")
                                 ))
        if colors != []:
            fig['data'][i + n_fill]['marker']['line']['color'] = colors[i]
            fig['data'][i + n_fill]['line']['color'] = colors[i]
    if vertical:
        fig.add_vrect(x0=vertical[0], x1=vertical[1], 
#               annotation_text="decline", annotation_position="top left",
              fillcolor="red", opacity=0.25, line_width=0)
        
    fig.update_xaxes(axis_style)
    fig.update_yaxes(axis_style)

    fig.update_layout(
        width = width,
        height = height,
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        yaxis = yaxis,
        xaxis = xaxis,
        showlegend=True
    )
    fig.update_layout(bg_style)

    fig.update_layout(legend=dict(
        bgcolor="White",
        bordercolor="Black",
        borderwidth=figborderlinesize
    ))
    fig.update_layout(font=dict(
        family="Times New Roman",
        size=20,
        color="Black"
    ))
    fig.update_layout(
        autosize=False,
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0,
            pad=0.1
        ),
        #     paper_bgcolor="LightSteelBlue",
    )
    fig.update_layout(legend=dict(
        yanchor=yanchor,
        y=y0_anchor,
        xanchor=xanchor,
        x=x0_anchor
    ))
    if len(xrange) == 2:
        fig.update_xaxes(range=xrange)
    if len(yrange) == 2:
        fig.update_yaxes(range=yrange)

    # fig.update_xaxes(type="log")
    # fig.update_yaxes(type="log")
    
    fig.show()
    fn = path + image_name
    print('Write image to file:', fn)
    fig.write_image(str(Path(fn)))
    # fig.write_image(str(Path(fn)), engine="kaleido")
    print("Successfully generated:", fn)

if __name__ == "__main__":
    path = '/Users/weugene/basilisk/work/tube/res27/'
    with open(path + "output_sliced_Rmax_U.json", "r") as f:
        outputs = json.load(f)
        times_del = [7.76]
        rm_i = []
        for t in times_del:
            rm_i.append(outputs['t'].index(t))

        for k, v in outputs.items():
            for i in rm_i:
                v.pop(i)
            outputs[k] = np.array(v)
    
    with open(path + "output_sliced_Rmax_U_cleaned.json", "r") as f:
        outputs2 = json.load(f)
        
    df = pd.read_csv(path + 'for_excel_table.txt', sep=' |\t', header=0)
    tmin = df['t'].values.min()
    tmax = df['t'].values.max()
    fn = "x_nose_mean_tip.png"
    plot_graph([df['t'].values, df['t'].values, outputs['t']], [df['x_nose'].values, df['x_mean'].values, outputs['x_tip']], \
               ['X nose', 'X mean', 'X tip'], \
               dash=['solid', 'dot', 'dash', 'dot'], \
               vertical=[7.34769,7.5],\
               xtitle="$t$", ytitle="$x$", image_name=fn[:-3]+'pdf', mode=['lines', 'lines', 'lines', 'lines'], \
               colors=['red', 'blue', 'black' ], \
               marker_size=1, width=1000, height=500, path=path, yanchor='top', y0_anchor=0.99, xanchor='left', x0_anchor=0.01)

#     fn = "u_nose_mean_estim_tip.png"
#     umean_estim = np.diff(df['x_mean'].values)/np.diff(df['t'].values)
#     plot_graph([df['t'].values, df['t'].values, outputs['t']], [np.diff(df['x_nose'].values)/np.diff(df['t'].values), umean_estim, outputs['U_tip']], \
#                ['U nose', 'U mean', 'U tip'], \
#                dash=['solid', 'dot', 'dash'], \
#                xtitle="t", ytitle="U", image_name=fn[:-3]+'pdf', mode=['lines', 'lines', 'lines'], \
#                colors=['red', 'blue', 'black' ], \
#                marker_size=1, width=1000, height=500, path=path, yanchor='bottom', y0_anchor=0.01, xanchor='left', x0_anchor=0.3)


    csvPattern = "slice_t=*.csv"
    csvnames = glob.glob(f'{path}/{csvPattern}', recursive = False)
    csvnames = sort_names(csvnames)
#     print('Found pvd files in:',csvnames)

    list_t = []
    list_unose = []
    # list_umean = []

    for ifile, file in enumerate(csvnames):
        time = give_time(file)
#         print(f'file: {file}, time={time}')
        res = pd.read_csv(path + file, sep = ',', usecols=['Points_0','Points_1','u.x_0','u.x_1','u.x_2','u.x_Magnitude'])
        row = res.iloc[res['Points_0'].idxmax()]
        list_unose.append(row['u.x_0'])
        list_t.append(float(time))
    list_unose = np.asarray(list_unose)
    list_t = np.asarray(list_t)
    
    
    fn = "u_nose_mean_tip_markers.png"
    t_limit=8.4
    tt = np.asarray(outputs['t'])
    args = tt < t_limit
    utip_x = tt[args]
    utip_y = np.asarray(outputs['U_tip'])[args]
    plot_graph([list_t, df['t'].values, utip_x], [list_unose, df['UmeanV'].values, utip_y], \
               ['$U_{\mathrm{nose}}$', '$U_{\mathrm{mean}}$', '$U_{\mathrm{tip}}$'], \
               yrange=[1,2],\
               vertical=[7.34769,7.5],\
               dash=['solid', 'solid', 'solid'], \
               xtitle="$t$", ytitle="$U$", image_name=fn[:-3]+'pdf', mode=['lines+markers', 'lines+markers', 'lines+markers'], \
               colors=['red', 'blue', 'black' ], \
               marker_size=5, width=1000, height=500, path=path, yanchor='bottom', y0_anchor=0.01, xanchor='right', x0_anchor=0.99, logx=False, logy=False)

    # list_unose=np.asarray(list_unose)
    # fn = "u_nose_markers.png"
    # plot_graph([list_t, outputs['t']], [list_unose - 1.6, outputs['U_tip']], \
    #            ['$U_{\\text{nose}} - \\hat{U}$'], \
    #            dash=['solid', 'dot', 'dot'], \
    #            xtitle="t", ytitle='$U_{\\text{nose}} - \\hat{U}$', image_name=fn[:-3]+'pdf', mode=['lines+markers', 'lines+markers', 'lines+markers'], \
    #            colors=['red', 'black', 'black' ], \
    #            marker_size=5, width=1000, height=500, path=path, yanchor='top', y0_anchor=0.99, xanchor='right', x0_anchor=0.99, logx=False, logy=False)


    # print(list_t, list_unose)
    fn = "u_nose_mean_markers_log.png"
    plot_graph([list_t, df['t'].values, outputs['t']], [list_unose, df['UmeanV'].values, outputs['U_tip']], \
#                ['U nose', 'U mean', 'U tip'], \
               ['$U_{\mathrm{nose}}$', '$U_{\mathrm{mean}}$', '$U_{\mathrm{tip}}$'], \
               dash=['solid', 'solid', 'solid'], \
               vertical=[7.34769,7.5],\
               xtitle="$t$", ytitle="$U$", image_name=fn[:-3]+'pdf', mode=['lines+markers', 'lines+markers', 'lines+markers'], \
               colors=['red', 'blue', 'black' ], \
               marker_size=5, width=1000, height=500, path=path, yanchor='bottom', y0_anchor=0.01, xanchor='left', x0_anchor=0.01, logx=True, logy=True)

    # list_unose=np.asarray(list_unose)
    # fn = "u_nose_markers_log.png"
    # plot_graph([list_t], [list_unose - 1.6], \
    #            ['$U_{\\text{nose}} - \\hat{U}$'], \
    #            dash=['solid', 'dot', 'dot'], \
    #            xtitle="t", ytitle='$U_{\\text{nose}} - \\hat{U}$', image_name=fn[:-3]+'pdf', mode=['lines+markers', 'lines+markers', 'lines+markers'], \
    #            colors=['red', 'black', 'black' ], \
    #            marker_size=5, width=1000, height=500, path=path, yanchor='top', y0_anchor=0.99, xanchor='right', x0_anchor=0.99, logx=True, logy=True)


Plot curve number: 0
Plot curve number: 1
Plot curve number: 2


/Users/weugene/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:172: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



Write image to file: /Users/weugene/basilisk/work/tube/res27/x_nose_mean_tip.pdf
Successfully generated: /Users/weugene/basilisk/work/tube/res27/x_nose_mean_tip.pdf
Time: 0.17413 -- 11.6057
Plot curve number: 0
Plot curve number: 1
Plot curve number: 2


Write image to file: /Users/weugene/basilisk/work/tube/res27/u_nose_mean_tip_markers.pdf
Successfully generated: /Users/weugene/basilisk/work/tube/res27/u_nose_mean_tip_markers.pdf
Plot curve number: 0
Plot curve number: 1
Plot curve number: 2


Write image to file: /Users/weugene/basilisk/work/tube/res27/u_nose_mean_markers_log.pdf
Successfully generated: /Users/weugene/basilisk/work/tube/res27/u_nose_mean_markers_log.pdf


In [2]:
ind = list_t > 2
mean_nose=statistics.mean(list_unose[ind])
stdv_nose=statistics.stdev(list_unose[ind])
mean_nose, stdv_nose, stdv_nose/mean_nose

(1.618707512953368, 0.004595303765699289, 0.00283887220447569)

In [3]:
ind = (df['t'].values > 7.45) &(df['t'].values < 8)
mean_mean=statistics.mean(df['UmeanV'].values[ind])
stdv_mean=statistics.stdev(df['UmeanV'].values[ind])
mean_mean, stdv_mean, stdv_mean/mean_mean

(1.5155051803913608, 0.001204862625293662, 0.0007950237589966672)

In [24]:
fn = "u_nose_mean_tip_markers2.png"
plot_graph([list_t, df['t'].values, outputs['t']], [list_unose, df['UmeanV'].values, outputs['U_tip']], \
#            ['U nose', 'U mean', 'U tip'], \
           ['$U_{\mathrm{nose}}$', '$U_{\mathrm{mean}}$', '$U_{\mathrm{tip}}$'], \
           dash=['solid', 'solid', 'solid'], \
           vertical=[7.34769,7.5],\
           xtitle="$t$", ytitle="$U$", image_name=fn[:-3]+'pdf', mode=['lines+markers', 'lines+markers', 'lines+markers'], \
           colors=['red', 'blue', 'black' ], \
           marker_size=5, width=1000, height=500, path=path, yanchor='top', y0_anchor=0.99, xanchor='right', x0_anchor=0.99, logx=False, logy=False)


Plot curve number: 0
Plot curve number: 1
Plot curve number: 2


Write image to file: /Users/weugene/basilisk/work/tube/res27/u_nose_mean_tip_markers2.pdf
Successfully generated: /Users/weugene/basilisk/work/tube/res27/u_nose_mean_tip_markers2.pdf


In [12]:
fn = "curvature_tip.png"
plot_graph([outputs['t']], [outputs['curvature_tip']], \
           ['$\kappa$'], \
           dash=['solid', 'dot', 'dash'], \
           xtitle="t", ytitle="$\kappa$", image_name=fn[:-3]+'pdf', mode=['lines+markers', 'lines+markers', 'lines+markers'], \
           colors=['red', 'black', 'black' ], \
           marker_size=5, width=1000, height=500, path=path, yanchor='top', y0_anchor=0.99, xanchor='right', x0_anchor=0.99, logx=False, logy=False)


Plot curve number: 0


Write image to file: /Users/weugene/basilisk/work/tube/res27/curvature_tip.pdf
Successfully generated: /Users/weugene/basilisk/work/tube/res27/curvature_tip.pdf


In [92]:
fn = "rmax.png"
ind = (outputs['t'] > 5.56) & (outputs['t'] < 7.13)
slope, intercept, r, p, se = linregress(outputs['t'][ind], outputs['rmax'][ind])
print(f"slope={slope} intercept={intercept}")
xx = np.linspace(outputs['t'][ind].min(), outputs['t'][ind].max())
yy = slope*xx + intercept
plot_graph([outputs['t'], xx], [outputs['rmax'], yy], \
           ['$R_{\max}$', f'y={slope:0.3f} t+{intercept:0.2f}'], \
           dash=['solid', 'solid', 'dash'], \
           xtitle="t", ytitle="$R_{\max}$", image_name=fn[:-3]+'pdf', mode=['lines+markers', 'lines', 'lines+markers'], \
           colors=['red', 'black', 'black' ], \
           marker_size=5, width=1000, height=500, path=path, yanchor='top', y0_anchor=0.99, xanchor='right', x0_anchor=0.99, logx=False, logy=False)


slope=0.03698093728506057 intercept=0.06951456512756066
Plot curve number: 0
Plot curve number: 1


Write image to file: /Users/weugene/basilisk/work/tube/res27/rmax.pdf
Successfully generated: /Users/weugene/basilisk/work/tube/res27/rmax.pdf


In [ ]:
{"t": [1.10827, 1.27861, 1.62507, 2.27704], "curvature_tip": [11.69990077360389, 10.46155854043738, 8.129304327606713, 9.484951543097077], "U_tip": [1.60205, 1.58437, 1.57448, 1.56208], "x_tip": [0.8362500000000002, 0.9382700000000002, 1.1131600000000001, 1.4290000000000003], "rmax": [0.335697, 0.318533, 0.251163, 0.259031]}

In [7]:
import plotly.graph_objects as go

fig = go.Figure()

ind = (outputs['t'] > 5.29) & (outputs['t'] < 7.58)
slope, intercept, r, p, se = linregress(outputs['t'][ind], outputs['curvature_tip'][ind])
print(f"slope={slope} intercept={intercept}")
xx = np.linspace(outputs['t'][ind].min(), outputs['t'][ind].max())
yy = slope*xx + intercept

ind = outputs['t'] < 7.81
fig.add_trace(go.Scatter(
    x=outputs['t'][ind],
    y=outputs['curvature_tip'][ind],
    name="$\kappa(t)$",
    mode='lines',
    marker=dict(
        size=5,
        line=dict(width=1)),
    marker_symbol='circle',
    line=dict(width=2, dash='solid', color='blue'),
    textfont=dict(
     family="Times New Roman",
     size=18,
     color="LightSeaGreen")
))
fig.add_trace(go.Scatter(
    x=xx,
    y=yy,
    name=f'$y={slope:0.3f}\cdot t+{intercept:0.2f}$',
    mode='lines',
    marker=dict(
        size=5,
        line=dict(width=1)),
    marker_symbol='circle',
    line=dict(width=2, dash='dash', color='black'),
    textfont=dict(
     family="Times New Roman",
     size=18,
     color="LightSeaGreen")
))

ind = (outputs['t'] > 5.56) & (outputs['t'] < 7.13)
slope, intercept, r, p, se = linregress(outputs['t'][ind], outputs['rmax'][ind])
print(f"slope={slope} intercept={intercept}")
xx = np.linspace(outputs['t'][ind].min(), outputs['t'][ind].max())
yy = slope*xx + intercept

ind = (outputs['t'] > 2 ) & (outputs['t'] < 7.8) #7.25
fig.add_trace(go.Scatter(
    x=outputs['t'][ind],
    y=outputs['rmax'][ind],
    name='$R_{\max}(t)$',
    yaxis="y2",
    mode='lines',
    marker=dict(
        size=5,
        line=dict(width=1)),
    marker_symbol='circle',
    line=dict(width=2, dash='solid', color='red'),
    textfont=dict(
     family="Times New Roman",
#      size=18,
     color="LightSeaGreen")
))

fig.add_trace(go.Scatter(
    x=xx,
    y=yy,
    name=f'$y={slope:0.3f}\cdot t+{intercept:0.2f}$',
    yaxis="y2",
    mode='lines',
    marker=dict(
        size=5,
        line=dict(width=1)),
    marker_symbol='circle',
    line=dict(width=2, dash='dot', color='black'),
    textfont=dict(
     family="Times New Roman",
     size=18,
     color="LightSeaGreen")
))
vertical=[7.34769,7.5]
fig.add_vrect(x0=vertical[0], x1=vertical[1], 
#               annotation_text="decline", annotation_position="top left",
              fillcolor="red", opacity=0.25, line_width=0)

vert_time=[7.34769, 7.40156, 7.42752, 7.51797]
for time in vert_time:
    fig.add_vline(x=time, line_width=1, line_dash="dash", line_color="grey")
# Create axis objects
fig.update_layout(
    xaxis=dict(
        title='$t$',
        domain=[0.7, 0.3]
    ),
    yaxis=dict(
        title="$\kappa$",
        titlefont=dict(
            color="blue", 
#             size=10
        ),
        tickfont=dict(
            color="blue"
        )
    ),
    yaxis2=dict(
        title="$R_{\mathrm{max}}$",
        titlefont=dict(
            color="red", 
#             size=10
        ),
        tickfont=dict(
            color="red"
        ),
        anchor="x",
        overlaying="y",
        side="right",
#         position=0.1
    ),
)
fig.update_layout(
    autosize=False,
    margin=dict(
        l=0.2,
        r=0.2,
        b=0.2,
        t=0.2,
        pad=0.1
    ),
    #     paper_bgcolor="LightSteelBlue",
)
fig.update_layout(legend=dict(
    yanchor='bottom',
    y=0.01,
    xanchor='center',
    x=0.5
))


figborderlinesize = 0.7
legborderlinesize = 0.7
axis_style = dict(showline=True, gridwidth=1, gridcolor='lightgrey', 
                  linewidth=figborderlinesize, 
                  linecolor='black', 
                  mirror=True,
                  ticks='outside', 
                  tickfont = dict(family = 'Times New Roman', size = 14, color = 'black'),
                  titlefont = dict(family = 'Times New Roman', size = 20, color = 'black')
            )

bg_style = {'plot_bgcolor': 'rgba(255, 255, 255, 1)', 'paper_bgcolor': 'rgba(255, 255, 255, 1)',}
fig.update_layout(bg_style, showlegend=True)
fig.update_xaxes(axis_style)
fig.update_yaxes(axis_style)
# Update layout properties
fig.update_layout(
    width=800,
)

fig.show()
fn = path + 'Rmax_curvature.pdf'
print('Write image to file:', fn)
fig.write_image(str(Path(fn)))

slope=-1.0443076469721542 intercept=14.648879079001363
slope=0.03698093728506057 intercept=0.06951456512756066


Write image to file: /Users/weugene/basilisk/work/tube/res27/Rmax_curvature.pdf
